## error対処後
- OSError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [1]:
#pip install matplotlib

In [2]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools
import random
import matplotlib.pyplot as plt
import timeit

In [3]:
#!pip3 install dwave-system

In [4]:
from dwave.system.composites import AutoEmbeddingComposite
from dwave.system.samplers import DWaveSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo

In [5]:
DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09" #"replace this string with actual token"

### 解析用データ作成

（3年間骨肉腫に罹患しない）事象と各特徴量とのP値を計算する。
- Osteogenic sarcoma data:骨肉腫
- LI:リンパ球浸出
- AOP:類骨病理学

In [6]:
import pandas as pd
import numpy as np

In [7]:
ost30 = pd.read_csv("../../input/ost30.csv", sep=',', index_col=0)
ost30.shape

(30, 4)

In [8]:
ost30.head()

,Y,LI,SEX,AOP
0,1,1,1,1
1,0,1,1,0
2,1,0,0,0
3,1,0,0,0
4,1,0,1,0


In [9]:
t_list = []
t_list.append(sum(ost30['Y']))
t_list.append(np.dot(ost30['Y'], ost30['LI']))
t_list.append(np.dot(ost30['Y'], ost30['SEX']))
t_list.append(np.dot(ost30['Y'], ost30['AOP']))

In [10]:
t_list#Y, LI, SEX, AOP

[18, 11, 5, 7]

In [11]:
columns = list(ost30.columns)
num_samples = ost30.shape[0]

### annealing_time = 20, num_reads=100, for t1 in range(0, ost16.shape[0]+1)

In [12]:
#dw_sampler = DWaveSampler()

#error発生

In [13]:
dw_sampler = DWaveSampler(
    endpoint="https://cloud.dwavesys.com/sapi",
    solver = 'DW_2000Q_6',
    token = DWAVE_TOKEN
)

In [14]:
qa_sampler = AutoEmbeddingComposite(dw_sampler)

In [15]:
def find_valid_y(df, num_reads):
    valid_y_list = {}
    valid_y_num = {}
    for t1 in range(0, df.shape[0]+1):
        y = Array.create('y', shape=df.shape[0], vartype='BINARY')

        #QUBO式で定式化
        H = (sum(y) - t_list[0])**2
        H += (np.dot(df[columns[1]], y) - t1)**2
        for j in range(2, len(columns)):
            H_plus = (np.dot(df[columns[j]], y) - t_list[j])**2
            H += H_plus

        qb = H.compile().to_qubo()
        res = qa_sampler.sample_qubo(qb[0], num_reads=100)

        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        for y_info in list(res.record):
            if sum(y_info[0]) == t_list[0]:#int同士の比較
                y = pd.Series(y_info[0])
                if np.dot(df[columns[1]], y) == t1:#int同士の比較
                    for j in range(2, len(columns)):
                        if np.dot(df[columns[j]], y) != t_list[j]:#int同士の比較
                            break
                    else:
                        if all(list(y_info[0]) != p for p in valid_y_list[t1]):
                            valid_y_num[t1] += 1
                            valid_y_list[t1].append(list(y_info[0]))
                            print('perfect')
    return valid_y_list, valid_y_num

In [16]:
valid_y_list, valid_y_num = find_valid_y(ost30, 100)
print(valid_y_list)
print('--------')
print(valid_y_num)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: []}
--------
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0}


In [17]:
loop = 10
df = ost30
num_reads = 100
result = timeit.timeit('find_valid_y(df, num_reads)', globals=globals(), number=loop)
print(result / loop, '秒')

313.9163353518 秒


In [20]:
print(result / loop/60, '分')

5.23193892253 分


In [18]:
valid_y_num

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0}

In [19]:
valid_y_num[9]/sum(valid_y_num.values())

ZeroDivisionError: division by zero